# Pulling data from Riot APIs

In [1]:
!pip install requests
!pip install pandas

import requests
import json
import pandas as pd


## Create Printing Functions

In [2]:
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)
    
def print_response(response):
    print(response.status_code)
    jprint(response.json())

## Set Hardcoded Variables

In [3]:
riotpath = "https://na1.api.riotgames.com"
americapath = "https://AMERICAS.api.riotgames.com"
apikey = "********"
ign = "Coconut in Pants"

## Get Summoner Info

In [4]:
summoner = requests.get("{}/tft/summoner/v1/summoners/by-name/{}?api_key={}".format(riotpath, ign, apikey))

summonerdata = json.loads(json.dumps(summoner.json(), indent = 4))

jprint(summonerdata)

{
    "accountId": "CbZARAYDhdCn-MTVaxukySAjiSTgvz3zRVdOxA1FHxph-0A",
    "id": "DgtH9M0NSZNCq-rpzy5f5S9NVCBByRBu37Kbhrc7d69hYxM",
    "name": "Coconut in Pants",
    "profileIconId": 3233,
    "puuid": "iaDL8kS51__kVTc1PWpJSeHwg4MgCZVovCg-7eohUNAD4pjSnAtQs9uSzWarikIG4K0YxZRnPon1sg",
    "revisionDate": 1686109951840,
    "summonerLevel": 267
}


## Get Match Ids from Summoner

In [5]:
matchids = requests.get("{}/tft/match/v1/matches/by-puuid/{}/ids?count=50&api_key={}".format(americapath, summonerdata['puuid'], apikey))
matchlist = json.loads(json.dumps(matchids.json(), indent = 4))
jprint(matchlist)

[
    "NA1_4676823326",
    "NA1_4676759875",
    "NA1_4676593034",
    "NA1_4676541629",
    "NA1_4676530469",
    "NA1_4675851574",
    "NA1_4675835907",
    "NA1_4675805427",
    "NA1_4675781510",
    "NA1_4675757332",
    "NA1_4675408449",
    "NA1_4675206613",
    "NA1_4674918300",
    "NA1_4674897301",
    "NA1_4674867326",
    "NA1_4674293536",
    "NA1_4673927582",
    "NA1_4673909162",
    "NA1_4673361634",
    "NA1_4673336559",
    "NA1_4673298390",
    "NA1_4673159804",
    "NA1_4673136125",
    "NA1_4673106324",
    "NA1_4673077159",
    "NA1_4673049549",
    "NA1_4673031291",
    "NA1_4672876930",
    "NA1_4672855413",
    "NA1_4672845988",
    "NA1_4672362023",
    "NA1_4672326556",
    "NA1_4672295132",
    "NA1_4672269127",
    "NA1_4672142499",
    "NA1_4672129136",
    "NA1_4672107067",
    "NA1_4672092836",
    "NA1_4671872749",
    "NA1_4671839023",
    "NA1_4671800414",
    "NA1_4671767073",
    "NA1_4671722623",
    "NA1_4671693489",
    "NA1_4671211938",
    "NA1

## Get Match Data and set as 'data'

In [6]:
match = requests.get("{}/tft/match/v1/matches/{}?api_key={}".format(americapath, matchlist[0], apikey))
data = json.loads(json.dumps(match.json(), indent = 4))


## Write Match data to a file

In [7]:
with open('results.json', 'w') as fp:
    json.dump(data, fp)


# Pushing data to Postgres

In [8]:
import psycopg2
import datetime

### Create a connection to the database and use the connection to get a cursor that can be used to execute queries

In [12]:
# Create connection to Database
def create_connection(dbname, user, password):
    try:
        conn = psycopg2.connect("host=127.0.0.1 dbname={} user={} password={}".format(dbname, user, password))
    except psycopg2.Error as e:
        print("Error: Could not make connection to the Postgres database")
        print(e)
   
    # Set automatic commit to be true so that each action is committed without having a call conn.commit() after each command
    conn.set_session(autocommit=True)

    #Use the connection to get a cursor that can be used to execute queries.
    try:
        cur = conn.cursor()
    except psycopg2.Error as e:
        print("Error: Could not get cursor to the Database")
        print(e) 

    return conn, cur

def execute_sql(sql, variables):
    try:
        cur.execute(sql, variables)
    except psycopg2.Error as e:
        print(e)

def epoch_to_date(epoch_time):
    time_stamp = epoch_time / 1000
    date_time = datetime.datetime.fromtimestamp(time_stamp).strftime('%Y-%m-%d %H:%M:%S')
    return date_time

### Create Functions for SQL Commnads

In [13]:
def drop_tables():
    commands = """DROP TABLE IF EXISTS matches CASCADE;
                  DROP TABLE IF EXISTS players_in_match CASCADE;
                  DROP TABLE IF EXISTS player_stats;
                  DROP TABLE IF EXISTS traits;
                  DROP TABLE IF EXISTS units CASCADE;
                  DROP TABLE IF EXISTS augments;
                  DROP TABLE IF EXISTS items;
               """
    try:
        cur.execute(commands)
        print("Dropped Tables: [matches, player_stats, traits, augments, units, players_in_match]")
    except psycopg2.Error as e:
        print("Error: Failed to drop tables: [matches, player_stats, traits, augments, units, players_in_match]")
        print(e)
        
def create_tables():
    commands = '''
    CREATE TABLE matches(
        m_id VARCHAR(20) NOT NULL,
        m_datetime BIGINT,
        m_length DECIMAL,
        PRIMARY KEY(m_id)
    );
    
    CREATE TABLE players_in_match(
        m_id VARCHAR(20) NOT NULL,
        puuid VARCHAR(100) NOT NULL,
        PRIMARY KEY(m_id, puuid),
        FOREIGN KEY (m_id) REFERENCES matches (m_id)
    );
    
    CREATE TABLE player_stats(
        m_id VARCHAR(20) NOT NULL,
        puuid VARCHAR(100) NOT NULL,
        gold_left INT, 
        last_round INT,
        level INT,
        placement INT,
        players_eliminated INT,
        time_eliminated FLOAT,
        total_damage_to_players INT,
        FOREIGN KEY (m_id, puuid) REFERENCES players_in_match (m_id, puuid),
        PRIMARY KEY(m_id, puuid)
    );
    
    CREATE TABLE traits(
        m_id VARCHAR(20) NOT NULL,
        puuid VARCHAR(100) NOT NULL,
        trait_name VARCHAR(100) NOT NULL,
        num_units INT,
        style INT,
        trait_tier_current INT,
        trait_tier_total INT,
        FOREIGN KEY (m_id, puuid) REFERENCES players_in_match (m_id, puuid),
        PRIMARY KEY(m_id, puuid, trait_name)
    );
    
    CREATE TABLE augments(
        m_id VARCHAR(20) NOT NULL,
        puuid VARCHAR(100) NOT NULL,
        augment VARCHAR(100) NOT NULL,
        pick INT,
        FOREIGN KEY (m_id, puuid) REFERENCES players_in_match (m_id, puuid),
        PRIMARY KEY(m_id, puuid, augment)
    );
    
    CREATE TABLE units(
        m_id VARCHAR(20) NOT NULL,
        puuid VARCHAR(100) NOT NULL,
        unit_id VARCHAR(100) NOT NULL,
        rarity INT,
        unit_tier INT,
        unit_copies INT,
        PRIMARY KEY(m_id, puuid, unit_id, rarity, unit_tier, unit_copies),
        FOREIGN KEY (m_id, puuid) REFERENCES players_in_match (m_id, puuid)
    );

    CREATE TABLE items(
        m_id VARCHAR(20) NOT NULL,
        puuid VARCHAR(100) NOT NULL,
        unit_id VARCHAR(100) NOT NULL,
        rarity INT,
        unit_tier INT,
        unit_copies INT,
        item VARCHAR(100) NOT NULL,
        FOREIGN KEY (m_id, puuid) REFERENCES players_in_match (m_id, puuid),
        FOREIGN KEY (m_id, puuid, unit_id, rarity, unit_tier, unit_copies) REFERENCES units (m_id, puuid, unit_id, rarity, unit_tier, unit_copies)
    );
    '''
    
    try:
        cur.execute(commands)
        print("Created tables: [matches, player_match_stats, player_match_traits, player_match_units]")
    except psycopg2.Error as e:
        print("Error: Failed to create tables: [matches, player_match_stats, player_match_traits, player_match_units]")
        print(e)



### Execute SQL Queries

In [14]:
conn, cur = create_connection("tft", "ichiu", "****")

drop_tables()
create_tables()


Dropped Tables: [matches, player_stats, traits, augments, units, players_in_match]
Created tables: [matches, player_match_stats, player_match_traits, player_match_units]


### Looping through matches provided from matchlist API and pushing matchdata to Postgres

In [15]:
matches_sql = '''
        INSERT INTO matches ("m_id", "m_datetime", "m_length") 
        VALUES (%s,%s,%s);'''
players_in_match_sql = '''
        INSERT INTO players_in_match ("m_id", "puuid") 
        VALUES (%s,%s);'''
players_stats_sql = '''
        INSERT INTO player_stats ("m_id", "puuid", "gold_left", "last_round", "level",
                "placement", "players_eliminated", "time_eliminated", "total_damage_to_players")
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s);'''
augments_sql = '''
        INSERT INTO augments ("m_id", "puuid", "augment", "pick") 
        VALUES (%s,%s,%s,%s);'''
traits_sql = '''
        INSERT INTO traits ("m_id", "puuid", "trait_name", "num_units", "style", 
                "trait_tier_current", "trait_tier_total")
        VALUES (%s,%s,%s,%s,%s,%s,%s);'''
units_sql = '''
        INSERT INTO units ("m_id", "puuid", "unit_id", "rarity", "unit_tier", "unit_copies")
        VALUES (%s,%s,%s,%s,%s,%s);'''
items_sql = '''
        INSERT INTO items ("m_id", "puuid", "unit_id", "rarity", "unit_tier", "unit_copies", "item") 
        VALUES (%s,%s,%s,%s,%s,%s,%s);'''

    
# Loop through match list
for i in range(len(matchlist)):
    match = requests.get("{}/tft/match/v1/matches/{}?api_key={}".format(americapath, matchlist[i], apikey))
    data = json.loads(json.dumps(match.json(), indent = 4))
    m_id = data["metadata"]["match_id"]
    p_list = data["info"]["participants"]
    
    # POPULATE MATCHES TABLE
    matches_vars = (m_id ,data["info"]["game_datetime"],data["info"]["game_length"])
    execute_sql(matches_sql, matches_vars)
        
    # Loop through list of participants
    for j in range(len(p_list)):
        
        # Set reusable variables at the participants level
        traits = p_list[j]["traits"]
        augments = p_list[j]["augments"]
        units = p_list[j]["units"]
        puuid = p_list[j]["puuid"]
        
        # POPULATE PLAYERS_IN_MATCH TABLE
        players_in_match_vars = (m_id, puuid)
        execute_sql(players_in_match_sql, players_in_match_vars)
        
        # POPULATE PLAYERS_STATS TABLE
        player_stats_vars = (m_id, puuid, p_list[j]["gold_left"], p_list[j]["last_round"], p_list[j]["level"], p_list[j]["placement"], p_list[j]["players_eliminated"], p_list[j]["time_eliminated"], p_list[j]["total_damage_to_players"])
        execute_sql(players_stats_sql, player_stats_vars)

        # Loop through augments and push to db
        for a in range(len(augments)): 
            augment_vars = (m_id, puuid, augments[a], a+1)
            execute_sql(augments_sql,augment_vars)
        
        # Loop through traits and push to db
        for t in range(len(traits)):
            trait_vars = (m_id, puuid, traits[t]["name"], traits[t]["num_units"], traits[t]["style"], traits[t]["tier_current"], traits[t]["tier_total"])
            execute_sql(traits_sql, trait_vars)
            
        # Loop through units and items and push to db
        unit_dup = []
        for u in range(len(units)):
            copy = 1
            unit_search = [m_id, puuid, units[u]["character_id"], units[u]["rarity"], units[u]["tier"]]
            if(unit_search in unit_dup):
                copy += 1
                unit_vars = (m_id, puuid, units[u]["character_id"], units[u]["rarity"], units[u]["tier"], copy)
                execute_sql(units_sql,unit_vars)
                for item in units[u]["itemNames"]:
                    item_vars = (m_id, puuid, units[u]["character_id"], units[u]["rarity"], units[u]["tier"], copy, item)
                    execute_sql(items_sql, item_vars)
            else:
                unit_vars = (m_id, puuid, units[u]["character_id"], units[u]["rarity"], units[u]["tier"], copy)
                execute_sql(units_sql,unit_vars)
                unit_dup.append(unit_search)
                for item in units[u]["itemNames"]:
                    item_vars = (m_id, puuid, units[u]["character_id"], units[u]["rarity"], units[u]["tier"], copy, item)
                    execute_sql(items_sql, item_vars)



# Transform Data

In [25]:
import pandas as pd


select_query_matches = '''SELECT * FROM matches LIMIT 10;'''
q_vars = ""
execute_sql(select_query_matches, q_vars)
matches_results = cur.fetchall()
list_matches=[]
for row in matches_results:
    current_match = [row[0], epoch_to_date(row[1]), int(row[2]/60)]
    list_matches.append(current_match)

df_matches = pd.DataFrame(list_matches, columns=['Match ID', 'Datetime', 'Length'])

#df_matches

#--------------------------------------------------------------------------------------------

select_query_players_stats = '''SELECT * FROM player_stats LIMIT 10;'''
q_vars = ""
execute_sql(select_query_players_stats, q_vars)
players_stats_results = cur.fetchall()

players_list=[]
for row in players_stats_results:
    current_player = [row[0], row[1], row[2], row[3], row[4], row[5], row[6], int(row[7]/60), row[8]]
    players_list.append(current_player)
    
df_players = pd.DataFrame(players_list, columns=['m_id', 'puuid', 'gold_left', 'last_round', 'level', 'placement', 'players_eliminated', 'time_eliminated', 'total_damage_to_players'])

#df_players
#--------------------------------------------------------------------------------------------

select_query_traits = '''SELECT * FROM traits LIMIT 40;'''
q_vars = ""
execute_sql(select_query_traits, q_vars)
traits_results = cur.fetchall()

traits_list=[]
for row in traits_results:
    trait_color = row[4]
    if row[4] == 0:
        trait_color = "Gray"
    elif row[4] == 1:
        trait_color = "Bronze"
    elif row[4] == 2:
        trait_color = "Silver"
    elif row[4] == 3:
        trait_color = "Gold"
    elif row[4] == 4:
        trait_color = "Prismatic"

    current_trait = [row[0], row[1], row[2], row[3], trait_color, row[5], row[6], row[5]/row[6]]
    traits_list.append(current_trait)
    
df_traits = pd.DataFrame(traits_list, columns=["m_id", "puuid", "trait_name", "num_units", "style", "trait_tier_current", "trait_tier_total", "trait_ratio"])

df_traits

,m_id,puuid,trait_name,num_units,style,trait_tier_current,trait_tier_total,trait_ratio
0,NA1_4676823326,KIWNG4DpBm8f3Y33vXbYgCuPtTnjrBnSrtF_Bxn072DzJR...,Set8_Ace,1,Bronze,1,2,0.500000
1,NA1_4676823326,KIWNG4DpBm8f3Y33vXbYgCuPtTnjrBnSrtF_Bxn072DzJR...,Set8_Admin,1,Gray,0,3,0.000000
2,NA1_4676823326,KIWNG4DpBm8f3Y33vXbYgCuPtTnjrBnSrtF_Bxn072DzJR...,Set8_Brawler,2,Bronze,1,4,0.250000
3,NA1_4676823326,KIWNG4DpBm8f3Y33vXbYgCuPtTnjrBnSrtF_Bxn072DzJR...,Set8_Channeler,1,Gray,0,4,0.000000
4,NA1_4676823326,KIWNG4DpBm8f3Y33vXbYgCuPtTnjrBnSrtF_Bxn072DzJR...,Set8_Deadeye,1,Gray,0,4,0.000000
5,NA1_4676823326,KIWNG4DpBm8f3Y33vXbYgCuPtTnjrBnSrtF_Bxn072DzJR...,Set8_Duelist,1,Gray,0,4,0.000000
6,NA1_4676823326,KIWNG4DpBm8f3Y33vXbYgCuPtTnjrBnSrtF_Bxn072DzJR...,Set8_Forecaster,1,Gold,1,1,1.000000
7,NA1_4676823326,KIWNG4DpBm8f3Y33vXbYgCuPtTnjrBnSrtF_Bxn072DzJR...,Set8_GenAE,1,Gray,0,2,0.000000
8,NA1_4676823326,KIWNG4DpBm8f3Y33vXbYgCuPtTnjrBnSrtF_Bxn072DzJR...,Set8_GunMage,3,Silver,2,3,0.666667
9,NA1_4676823326,KIWNG4DpBm8f3Y33vXbYgCuPtTnjrBnSrtF_Bxn072DzJR...,Set8_Heart,1,Gray,0,3,0.000000
